In [2]:
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pprint
import json
import os
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook as tqdm

In [ ]:
def spec(value,*args):
    i= specs[specs['event_id'] == value].index.values[-1]
    print('Index :',i)
    print('Event_code :',train[train['event_id'] == value]['event_code'].unique()[-1])
    for arg in args:
        if(arg == 'info'):
         print(specs[arg][i])
        elif(arg == 'args'):
         print(pprint.pprint(json.loads(specs[arg][i])))
        else:
         print('Nothing')

In [ ]:
def event(value):
    i = train[train['event_id'] == value].index.values[-1]
    print(pprint.pprint(json.loads(train['event_data'][i])))

In [7]:
def get_game_feature(df,data):
    g0_d = dict(df[df['type'] == 'Game'].groupby('installation_id')['type'].value_counts())
    g0 = {}
    for (k,i),v in g0_d.items():
        g0[k] = v
    del g0_d
    df['G0'] = df['installation_id'].map(g0)
    df['G0'].fillna(0,inplace=True)

    g1_d = dict(df[(df['event_code'] == 4020) & (df['type'] == 'Game')].groupby('installation_id')['event_code'].value_counts())
    g1 = {}
    for (k,i),v in g1_d.items():
        g1[k] = v
    del g1_d
    df['G1'] = df['installation_id'].map(g1)
    df['G1'].fillna(0,inplace=True)
    del g1
    
    df['contains_true'] = df[(df['event_code'] == 4020) & (df['type'] == 'Game')]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>= 0) else 0)
    g2 = dict(df[df['event_code'] == 4020].groupby('installation_id')['contains_true'].sum())
    df['G2'] = df['installation_id'].map(g2)
    df['G2'].fillna(0,inplace=True)
    for c in ['contains_true']:
        df.pop(c)
        
    df['G3'] = df['G2'] / df['G1']
    df['G3'].fillna(0,inplace=True)
    
    i = 4
    for d in tqdm(data):
        g_attempt_d = dict(df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)].groupby('installation_id')['title'].value_counts())
        g_attempt = {}
        for (k,z),v in g_attempt_d.items():
            g_attempt[k] = v

        df[f'G{i}_attempt'] = df['installation_id'].map(g_attempt)
        df[f'G{i}_attempt'].fillna(0,inplace=True)

        df['contains_true'] = df[(df['type'] == 'Game') & (df['event_code'] == 4020) & (df['title'] == d)]['event_data'].map(lambda x:1 if(str(x).find('"correct":true')>=0) else 0)
        g = dict(df.groupby('installation_id')['contains_true'].sum())
        df[f'G{i}_correct'] = df['installation_id'].map(g)
        df[f'G{i}_correct'].fillna(0,inplace=True)

        df[f'G{i}_accuracy'] = df[f'G{i}_correct'] / df[f'G{i}_attempt']
        df[f'G{i}_accuracy'].fillna(0,inplace=True)
        
        df
        
        for c in ['contains_true',f'G{i}_attempt']:
            df.pop(c)
            
        i = i + 1

In [3]:
train = pd.read_csv('/kaggle/input/data-science-bowl-2019/train.csv')
train_labels = pd.read_csv('/kaggle/input/data-science-bowl-2019/train_labels.csv')
specs = pd.read_csv('/kaggle/input/data-science-bowl-2019/specs.csv')
test = pd.read_csv('/kaggle/input/data-science-bowl-2019/test.csv')
submission = pd.read_csv('/kaggle/input/data-science-bowl-2019/sample_submission.csv')

In [24]:
assessment_id = list(train[train['type'] == 'Assessment']['installation_id'].unique())
train = train.loc[train['installation_id'].isin(assessment_id)]

In [25]:
train = train.reset_index(drop=False)
train.drop(columns = ['index'],axis = 1, inplace = True)

In [12]:
train[((train['event_code'] == 4100) & (train['title'] != 'Bird Measurer (Assessment)')) | ((train['event_code'] == 4110) & (train['title'] == 'Bird Measurer (Assessment)'))].head(200)

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world,G0,G1,G2,G3,G4_correct,G4_accuracy,G5_correct,G5_accuracy,G6_correct,G6_accuracy,G7_correct,G7_accuracy,G8_correct,G8_accuracy,G9_correct,G9_accuracy,G10_correct,G10_accuracy,G11_correct,G11_accuracy,G12_correct,G12_accuracy,G13_correct,G13_accuracy,G14_correct,G14_accuracy
2228,25fa8af4,901acc108f55a5a1,2019-08-06T05:22:32.357Z,"{""correct"":true,""stumps"":[1,2,4],""event_count""...",0006a69f,44,4100,31011,Mushroom Sorter (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2308,14de4c5d,80d34a30c2998653,2019-08-06T05:24:50.323Z,"{""distance"":10,""target_distances"":[5,6,7,8,9,1...",0006a69f,76,4100,114370,Air Show,Game,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2335,14de4c5d,80d34a30c2998653,2019-08-06T05:25:11.292Z,"{""distance"":9,""target_distances"":[5,6,7],""corr...",0006a69f,103,4100,135341,Air Show,Game,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2375,14de4c5d,80d34a30c2998653,2019-08-06T05:25:37.207Z,"{""distance"":3,""target_distances"":[5,6,7],""corr...",0006a69f,143,4100,161258,Air Show,Game,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2409,14de4c5d,80d34a30c2998653,2019-08-06T05:26:01.055Z,"{""distance"":8,""target_distances"":[5,6,7],""corr...",0006a69f,177,4100,185103,Air Show,Game,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2709,17113b36,77b8ee947eb84b4e,2019-08-06T05:35:54.898Z,"{""correct"":false,""caterpillars"":[11,8,3],""even...",0006a69f,29,4110,35771,Bird Measurer (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2715,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:01.927Z,"{""correct"":false,""caterpillars"":[11,8,11],""eve...",0006a69f,35,4110,42805,Bird Measurer (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2720,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:06.512Z,"{""correct"":false,""caterpillars"":[11,8,5],""even...",0006a69f,40,4110,47388,Bird Measurer (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2725,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:09.739Z,"{""correct"":false,""caterpillars"":[11,8,7],""even...",0006a69f,45,4110,50605,Bird Measurer (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000
2730,17113b36,77b8ee947eb84b4e,2019-08-06T05:36:13.951Z,"{""correct"":false,""caterpillars"":[11,8,4],""even...",0006a69f,50,4110,54822,Bird Measurer (Assessment),Assessment,TREETOPCITY,1732.0,171.0,118.0,0.690058,0.0,0.000000,18.0,0.400000,9.0,0.750000,0.0,0.000000,25.0,0.657895,23.0,0.920000,0.0,0.000000,0.0,0.000000,42.0,0.893617,1.0,0.25,0.0,0.000000


**For Game**

In [10]:
game_list = set(train[train['type'] == 'Game']['title'].unique()).union(set(test[test['type'] == 'Game']['title'].unique()))
game_list = list(game_list)

In [11]:
get_game_feature(train,game_list)

In [14]:
get_game_feature(test,game_list)

In [47]:
activities_list = set(train[train['type'] == 'Activity']['title'].unique()).union(set(test[test['type'] == 'Activity']['title'].unique()))
activities_list = list(activities_list)

8

In [13]:
train[pd.isnull(train).any(axis=1)]['installation_id'].unique()

array([], dtype=object)